In [7]:
%matplotlib inline
%load_ext autoreload

import sys
import csv
import multiprocessing

import ujson
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
import tensorflow as tf
import gensim
from gensim.models.fasttext import FastText
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
from tqdm import tqdm 
import seaborn as sns
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
csv.field_size_limit(500 * 1024 * 1024)

524288000

In [9]:
path_data = 'data/fake_news_corpus/'
path_news_csv = path_data + 'news_cleaned_2018_02_13.csv'
path_fasttext = path_data + 'news_cleaned_2018_02_13.fasttext.bin'
path_news_preprocessed = path_data + 'news_cleaned_2018_02_13.preprocessed.jsonl'
path_news_shuffled = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.jsonl'
path_news_train = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.train.jsonl'
path_news_test = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.test.jsonl'
path_news_val = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.val.jsonl'

path_news_embedded = path_data + 'news_cleaned_2018_02_13.embedded.jsonl'

# Load FastText

In [10]:
fasttext = FastText.load_fasttext_format(path_fasttext)

FileNotFoundError: [Errno 2] No such file or directory: 'data/fake_news_corpus/news_cleaned_2018_02_13.fasttext.bin'

# Load news_cleaned

In [ ]:
for df_news_chunk in pd.read_csv(path_news_csv, chunksize=1000):
    break

# Prepare the input data 

## Make sure the file is correct

In [ ]:
with tqdm() as progress:
    for df_news_chunk in pd.read_csv(path_news_csv, encoding='utf-8', engine='python', chunksize=1000):
        progress.update()

## News generator

In [ ]:
def news_generator():
    with tqdm() as progress:
        for df_news_chunk in pd.read_csv(path_news_csv, encoding='utf-8', engine='python', chunksize=10 * 1000):
            news_filter = df_news_chunk.type.isin(set(['fake', 'conspiracy', 'unreliable', 'reliable']))
            df_news_chunk_filtered = df_news_chunk[news_filter]
            for row in df_news_chunk_filtered.itertuples():
                label = 1 if row.type == 'reliable' else 0

                progress.update()
                yield int(row.id), '%s %s' % (row.title, row.content), label

## Show text length distribution

In [ ]:
lens = []
for i, (_id, con, label) in enumerate(news_generator()):
    if i > 10 * 1000:
        break

    lens.append(len(con))
    progress.update()

In [ ]:
len(lens), sum(lens) / len(lens), max(lens), min(lens)

In [5]:
sns.distplot(lens)

NameError: name 'lens' is not defined

## Preprocess  